In [93]:
import pandas as pd
import torch
import numpy as np
from datetime import datetime

In [94]:
data = pd.read_csv("../Current_Data/New_data_updated_names.csv", usecols=['Country','Date','Cases','Deaths','Summary'], 
                  parse_dates=["Date"])
data.head()

,Country,Date,Cases,Deaths,Summary
0,Benin,2014-01-01,11,0.0,Two districts crossed the alert threshold at t...
1,Burkina Faso,2014-01-01,88,13.0,Two districts crossed the alert threshold at t...
2,Ivory Coast,2014-01-01,3,2.0,Two districts crossed the alert threshold at t...
3,Ghana,2014-01-01,3,0.0,Two districts crossed the alert threshold at t...
4,Mali,2014-01-01,0,0.0,Two districts crossed the alert threshold at t...


In [101]:
country_list = ['Benin', 'Burkina Faso', 'Burundi', 'Cameroun', 'Centrafrique', "Ivory Coast", 'Ethiopia', 'Ghana',
               'Guinea', 'Guinea Bissau', 'Gambia', 'Kenya', 'Mali', 'Mauritania', 'Niger', 'Nigeria', 'Dem. Rep. Congo', 
                'Senegal' , 'South Sudan', 'Sudan', 'Sierra Lone', 'Tanzania', 'Chad', 'Togo', 'Uganda']
bordering_countries = {"Benin": ["Togo", "Nigeria", "Niger", "Burkina Faso"], 
                       "Burkina Faso": ["Mali", "Niger", "Benin" ,"Togo", "Ghana", "Ivory Coast"], 
                       "Burundi": ["Dem. Rep. Congo", "Tanzania"], 
                       "Cameroun": ["Centrafrique", "Nigeria", "Guinea", "Chad"], 
                       "Centrafrique": ["Cameroun", "Sudan", "South Sudan","Dem. Rep. Congo", "Chad"], 
                       "Ivory Coast": ["Ghana", "Mali", "Burkina Faso"], 
                       "Ethiopia": ["Sudan", "South Sudan", "Kenya"], 
                       "Ghana":["Guinea", "Ivory Coast", "Burkina Faso", "Togo"], 
                       "Guinea":["Ivory Coast", "Guinea Bissau", "Mali", "Senegal", "Sierra Lone"], 
                       "Guinea Bissau": ["Senegal", "Guinea"], 
                       "Gambia":["Senegal"], 
                       "Kenya": ["Ethiopia", "South Sudan", "Uganda"],
                       "Mali": ["Niger", "Burkina Faso", "Ivory Coast", "Guinea", "Mauritania", "Senegal"], 
                       "Mauritania":["Mali", "Senegal"], 
                       "Niger": ["Benin", "Nigeria", "Burkina Faso", "Mali", "Chad"], 
                       "Nigeria": ["Benin", "Cameroun", "Niger", "Chad"], 
                       "Dem. Rep. Congo": ["Burundi", "Centrafrique", "South Sudan", "Tanzania", "Uganda"], 
                       "Senegal":["Gambia", "Guinea", "Guinea Bissau", "Mali","Mauritania"], 
                       "South Sudan":["Sudan", "Ethiopia", "Kenya", "Dem. Rep. Congo", "Uganda","Centrafrique"], 
                       "Sudan":["Ethiopia", "South Sudan", "Centrafrique", "Chad"], 
                       "Sierra Lone": ["Guinea"], 
                       "Tanzania":["Kenya", "Uganda", "Burundi", "Dem. Rep. Congo"], 
                       "Chad":["Sudan", "Centrafrique", "Cameroun", "Niger", "Nigeria"], 
                       "Togo":["Ghana", "Burkina Faso","Benin", "Guinea"],
                       "Uganda":["Kenya", "South Sudan", "Dem. Rep. Congo", "Tanzania"]}
#this lambda function returns the index of the country from the corresponding country_list array
get_indices = lambda x: country_list.index(str(x).rstrip())
country_list.index("Guinea")

8

In [96]:
#creates a list of bordering countries for each datapoint with the countries represented as indices of country_list array
data_bordering_countries = pd.DataFrame(data["Country"].apply(lambda x: 
                                                              list(
                                                                  map(get_indices, 
                                                                      bordering_countries[str(x).rstrip()]))
                                                                   ))
data.insert(5, "Bordering Countries", data_bordering_countries)
#replace country names in country column with corresponding indices from country_list array
data["Country"] = data["Country"].apply(lambda x: country_list.index(str(x).rstrip()))
data.loc[0:20]

,Country,Date,Cases,Deaths,Summary,Bordering Countries
0,0,2014-01-01,11,0.0,Two districts crossed the alert threshold at t...,"[23, 15, 14, 1]"
1,1,2014-01-01,88,13.0,Two districts crossed the alert threshold at t...,"[12, 14, 0, 23, 7, 5]"
2,5,2014-01-01,3,2.0,Two districts crossed the alert threshold at t...,"[7, 12, 1]"
3,7,2014-01-01,3,0.0,Two districts crossed the alert threshold at t...,"[8, 5, 1, 23]"
4,12,2014-01-01,0,0.0,Two districts crossed the alert threshold at t...,"[14, 1, 5, 8, 13, 17]"
5,13,2014-01-01,0,0.0,Two districts crossed the alert threshold at t...,"[12, 17]"
6,15,2014-01-01,0,0.0,Two districts crossed the alert threshold at t...,"[0, 3, 14, 22]"
7,17,2014-01-01,3,0.0,Two districts crossed the alert threshold at t...,"[10, 8, 9, 12, 13]"
8,22,2014-01-01,7,0.0,Two districts crossed the alert threshold at t...,"[19, 4, 3, 14, 15]"
9,23,2014-01-01,7,0.0,Two districts crossed the alert threshold at t...,"[7, 1, 0, 8]"


In [66]:
def find_next_batch(count):
    initialCount = count
    found = False
    while not found:
        count += 1
        if count == len(data) - 1 or data["Date"].loc[count + 1] != data["Date"].loc[count]:
            found = True
            break
    return data.iloc[initialCount:count+1]

In [67]:
find_next_batch(0)

,Country,Date,Cases,Deaths,Summary,Bordering Countries
0,0,2014-01-01,11,0.0,Two districts crossed the alert threshold at t...,"[23, 15, 14, 1]"
1,1,2014-01-01,88,13.0,Two districts crossed the alert threshold at t...,"[12, 14, 0, 23, 7, 5]"
2,5,2014-01-01,3,2.0,Two districts crossed the alert threshold at t...,"[7, 12, 1]"
3,7,2014-01-01,3,0.0,Two districts crossed the alert threshold at t...,"[8, 5, 1, 23]"
4,12,2014-01-01,0,0.0,Two districts crossed the alert threshold at t...,"[14, 1, 5, 8, 13, 17]"
5,13,2014-01-01,0,0.0,Two districts crossed the alert threshold at t...,"[12, 17]"
6,15,2014-01-01,0,0.0,Two districts crossed the alert threshold at t...,"[0, 3, 14, 22]"
7,17,2014-01-01,3,0.0,Two districts crossed the alert threshold at t...,"[10, 8, 9, 12, 13]"
8,22,2014-01-01,7,0.0,Two districts crossed the alert threshold at t...,"[19, 4, 3, 14, 15]"
9,23,2014-01-01,7,0.0,Two districts crossed the alert threshold at t...,"[7, 1, 0, 8]"


In [77]:
find_next_batch(57)

,Country,Date,Cases,Deaths,Summary,Bordering Countries
57,5,2014-02-03,8,1.0,"At the week 6 of the 2014 epidemic season, thr...","[7, 12, 1]"
58,7,2014-02-03,10,2.0,"At the week 6 of the 2014 epidemic season, thr...","[8, 5, 1, 23]"
59,8,2014-02-03,0,0.0,"At the week 6 of the 2014 epidemic season, thr...","[5, 9, 12, 17, 20]"
60,10,2014-02-03,0,0.0,"At the week 6 of the 2014 epidemic season, thr...",[17]
61,12,2014-02-03,7,0.0,"At the week 6 of the 2014 epidemic season, thr...","[14, 1, 5, 8, 13, 17]"
62,13,2014-02-03,0,0.0,"At the week 6 of the 2014 epidemic season, thr...","[12, 17]"
63,14,2014-02-03,10,1.0,"At the week 6 of the 2014 epidemic season, thr...","[0, 15, 1, 12, 22]"
64,15,2014-02-03,0,0.0,"At the week 6 of the 2014 epidemic season, thr...","[0, 3, 14, 22]"
65,17,2014-02-03,4,0.0,"At the week 6 of the 2014 epidemic season, thr...","[10, 8, 9, 12, 13]"
66,19,2014-02-03,5,0.0,"At the week 6 of the 2014 epidemic season, thr...","[6, 18, 4, 22]"


In [73]:
data.loc[0:]

,Country,Date,Cases,Deaths,Summary,Bordering Countries
2806,1,2019-12-16,87,1.0,"From week 49 to 52 (December 2019), 8 district...","[12, 14, 0, 23, 7, 5]"
2807,2,2019-12-16,6,0.0,"From week 49 to 52 (December 2019), 8 district...","[16, 21]"
2808,4,2019-12-16,13,3.0,"From week 49 to 52 (December 2019), 8 district...","[3, 19, 18, 16, 22]"
2809,5,2019-12-16,9,0.0,"From week 49 to 52 (December 2019), 8 district...","[7, 12, 1]"
2810,7,2019-12-16,34,2.0,"From week 49 to 52 (December 2019), 8 district...","[8, 5, 1, 23]"
2811,8,2019-12-16,33,1.0,"From week 49 to 52 (December 2019), 8 district...","[5, 9, 12, 17, 20]"
2812,10,2019-12-16,0,0.0,"From week 49 to 52 (December 2019), 8 district...",[17]
2813,12,2019-12-16,44,0.0,"From week 49 to 52 (December 2019), 8 district...","[14, 1, 5, 8, 13, 17]"
2814,13,2019-12-16,0,0.0,"From week 49 to 52 (December 2019), 8 district...","[12, 17]"
2815,14,2019-12-16,17,3.0,"From week 49 to 52 (December 2019), 8 district...","[0, 15, 1, 12, 22]"


In [100]:
def update_batch(temp_batch, curr_batch, row):
    for index, row in curr_batch.iterrows():
            #check if the country is in the temp monthly batch, if not add it, otherwise update Cases, Deaths, and Summary
            if row["Country"] not in temp_batch["Country"].unique():
                if len(temp_batch) != 0:
                    row["Date"] = temp_batch["Date"].iloc[0]
                temp_batch = temp_batch.append(row)
            else:
                temp_batch.loc[temp_batch.Country == row["Country"], "Cases"] += row["Cases"]
                temp_batch.loc[temp_batch.Country == row["Country"], "Deaths"] += row["Deaths"]
                temp_batch.loc[temp_batch.Country == row["Country"], "Summary"] += "." + row["Summary"]
    return temp_batch
    
#Convert the weekly data into montly data (by summing over the Cases, Deaths, and concatenating the summaries)
count = 0
new_monthly_data = pd.DataFrame(columns=["Country", "Date", "Cases", "Deaths", "Summary", "Bordering Countries"])
#create a temporary dataframe for current month which will be concatenated to the final dataframe
temp_batch = new_monthly_data.copy()
curr_month = 1
#The upper-bound is determined by the number which when passed to the find_next_batch function will return the final batch
while count <= 2806:
    curr_batch = find_next_batch(count)
    count += len(curr_batch)
    #if the new batch is of a different month, add it to the new_monthly_data dataframe and empty the temp_batch for the new month
    if curr_batch["Date"].iloc[0].month != curr_month:
        curr_month = curr_batch["Date"].iloc[0].month
        new_monthly_data = pd.concat([new_monthly_data, temp_batch], ignore_index = True)
        temp_batch = pd.DataFrame(columns=["Country", "Date", "Cases", "Deaths", "Summary", "Bordering Countries"])
    temp_batch = update_batch(temp_batch, curr_batch, row)

new_monthly_data.to_csv("../Current_Data/Month_data.csv")

    

2014-01-01 00:00:00
0
1
7
12
13
15
17
22
23
0
1
5
7
13
15
17
22
23
0
1
5
7
8
13
15
17
22
23
0
1
5
7
8
12
13
15
17
22
23
2014-02-03 00:00:00
0
1
7
8
10
13
14
15
17
19
23
0
1
4
5
7
8
12
13
15
17
18
0
1
4
5
7
8
10
12
13
14
15
16
17
18
19
22
23
2014-03-03 00:00:00
0
1
4
5
7
10
12
13
15
17
18
19
22
23
0
1
4
5
7
8
10
12
13
15
16
17
18
19
22
0
1
3
7
10
12
13
14
15
16
17
18
19
23
0
1
3
4
5
7
8
10
12
13
14
15
17
18
19
23
2014-04-07 00:00:00
0
1
5
7
8
10
12
13
14
15
17
19
0
1
5
7
8
10
12
13
15
17
19
0
1
3
5
7
8
10
12
13
14
15
17
19
2014-05-05 00:00:00
0
1
4
5
7
8
10
15
17
23
0
1
5
7
8
10
13
15
16
17
19
23
0
5
7
10
13
15
17
18
19
22
23
2014-06-02 00:00:00
0
1
3
4
5
7
8
13
15
17
19
23
0
1
3
5
8
10
13
14
15
17
19
23
0
1
5
8
10
13
14
15
17
19
23
2014-07-18 00:00:00
2014-08-18 00:00:00
2014-09-15 00:00:00
2014-10-16 00:00:00
2014-11-17 00:00:00
2014-12-15 00:00:00
0
1
4
5
7
8
10
12
13
14
19
23
2015-01-05 00:00:00
0
4
5
8
10
12
13
14
15
19
0
1
4
5
7
8
10
12
13
14
16
17
18
19
23
0
1
3
5
8
10
12
13
14
1